# Emily coati

Make dataframe from Ben's feeding experiment data

In [24]:
import pandas as pd
import os
import numpy as np
import pickle
import librosa
import datetime
import glob

In [2]:
PROJECT_FOLDER = "/Volumes/EAS_ind/egrout/data/Ben's feeding experiment/"
PROJECT_FOLDER = "/Volumes/EAS_ind/egrout/data/Ben's feeding experiment_updated/"

In [77]:
def get_time(timestring):    
    time_patterns = ['%H:%M:%S.%f', '%M:%S.%f']    
    if(timestring)==0:
        return datetime.datetime.strptime('0:00:00.00', '%H:%M:%S.%f')
    else:
        for pattern in time_patterns:
            try:
                return datetime.datetime.strptime(timestring, pattern)
            except:
                pass
    
        print("Date is not in expected format") 
        print(timestring)
        sys.exit(0)

def get_s(dt):
    return dt.microsecond/1000000+dt.second + dt.minute*60 + dt.hour*60*60

def get_audio(wav_loc, start_s, duration_s):
    data = np.asarray([0.])
    if wav_loc not in fail_dict.keys():
        try:
            data, rate = librosa.load(wav_loc, offset=start_s, duration=duration_s, sr=None)        
            if np.issubdtype(type(data[0]), np.integer):
                data = data.astype('float32') 
                if rate != 48000:
                    print(rate, 'Hz for ', wav_loc)
        except Exception:
            print('failed_read for ', wav_loc)
            fail_dict[wav_loc] = "failed"
    return data

# Generate one labelfile for all data, including audio

In [71]:
project_ids = os.listdir(PROJECT_FOLDER)
subfolders = [PROJECT_FOLDER+x+"/" for x in project_ids]

In [ ]:
# too many mistakes in folder namings...

In [56]:
count=0
for project_id, subfolder in zip(project_ids, subfolders):
    csv_files = glob.glob(subfolder+'*.csv')
    if len(csv_files)==1:
        csv_file = csv_files[0]   
        labelfile = pd.read_csv(csv_file, sep="\t")    
        count=count+labelfile.shape[0]
    else:
        print("No or multiple csvs for ", project_id)
print(count)   

No or multiple csvs for  6.21.2004.PQ.d7a
7678


In [115]:
#all_labelfiles = []
#fail_dict = {}

for project_id, subfolder in zip(missing_project_ids, missing_subfolders):
    print("Extracting audio for ", project_id)
    
    # check if only one csv is there
    csv_files = glob.glob(subfolder+'*.csv')
    if len(csv_files)==1:
        csv_file = csv_files[0]
        
        # check if only one wav is there
        wav_files = glob.glob(subfolder+'*.wav')
        if len(wav_files)==1:
            wav_file = wav_files[0]

            # then I assume these are a valid pair and read in data
            labelfile = pd.read_csv(csv_file, sep="\t") 
            print(labelfile.shape[0], " calls in file")
            
            # make adaptations
            labelfile['start_s'] = labelfile.apply(lambda row: get_s(get_time(row['Start'])), axis=1)
            labelfile['duration_s'] = labelfile.apply(lambda row: get_s(get_time(row['Duration'])), axis=1)
            labelfile['callID'] = [project_id+'_'+str(x) for x in labelfile.start_s]

            # read audio
            raw_audio = labelfile.apply(lambda row: get_audio(wav_file, # location of long audio file
                                                              row['start_s'], # start of call in that file
                                                              row['duration_s']), # duration of call
                                        axis=1)

            labelfile['raw_audio'] = raw_audio
            all_labelfiles.append(labelfile)
        else:
            print("No or multiple wav for ", project_id)
    else:
        print("No or multiple csv for ", project_id)

Extracting audio for  7.28.2003.PS.20
221  calls in file
Extracting audio for  11.20.2003.PS.10a
39  calls in file
failed_read for  /Volumes/EAS_ind/egrout/data/Ben's feeding experiment_updated/11.20.2003.PS.10a/11.20.2003.PS.10a.wav
Extracting audio for  7.20.2004.PS.50b
20  calls in file
Extracting audio for  8.29.2004.PS.100b
24  calls in file
Extracting audio for  6.25.2004.PS.d7
173  calls in file
Extracting audio for  8.25.2004.PS.d2
157  calls in file
Extracting audio for  5.6.2004.PQ.d7
190  calls in file
Extracting audio for  8.19.2003.PQ.d2
99  calls in file
Extracting audio for  6.2.2004.PS.2
84  calls in file
Extracting audio for  5.30.2004.PQ.d2complete
80  calls in file
Extracting audio for  3.2.2004.PQ.d2
26  calls in file
Extracting audio for  9.16.2003.PS.20
158  calls in file
Extracting audio for  9.16.2004.PS.100
212  calls in file
Extracting audio for  7.4.2004.PS.10
3  calls in file
Extracting audio for  11.27.2003.PS.d7
210  calls in file
failed_read for  /Volumes

In [116]:
full_labelfile = pd.concat(all_labelfiles)
full_labelfile.shape

In [123]:
full_labelfile = full_labelfile.drop_duplicates(subset=['callID'])
full_labelfile.to_pickle('/Volumes/MaraMeerkat/coati_feeding.pkl')

In [124]:
full_labelfile.shape

(7671, 10)

In [ ]:
# Check what is missing

In [126]:
project_id = [x.split('_')[0] for x in full_labelfile.callID]
full_labelfile['project_id'] = project_id

In [148]:
new_callIDs = [x.replace('9.14.2003.PS.10compGH','9.14.2003.PS.10complGH') for x in full_labelfile.callID]
new_callIDs = [x.replace('10.8.2003.PG.d3','10.8.2003.PQ.d3') for x in new_callIDs]
full_labelfile['callID'] = new_callIDs

/Users/marathomas/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [130]:
ids_in_labelfile = list(set(full_labelfile.project_id))
len(ids_in_labelfile)

60

In [147]:
ids_in_labelfile

['5.23.2004.PS.20',
 '9.7.2004.PS.100',
 '9.14.2003.PS.20',
 '3.19.2004.PQ.d3',
 '7.28.2003.PS.20',
 '6.22.2004.PQ.100a',
 '6.27.2004.PS.d3',
 '9.29.2004.PS.d3',
 '7.4.2004.PS.d7',
 '6.25.2004.PS.d7',
 '6.14.2004.PQj.d7',
 '11.20.2003.PS.10b',
 '7.20.2004.PS.50b',
 '11.20.2003.PS.10a',
 '8.12.2003.PS.d3',
 '8.29.2004.PS.100b',
 '9.30.2004.PS.d7',
 '9.15.2003.PQ.20',
 '11.27.2003.PS.50',
 '9.200.2003.PQ.20',
 '11.27.2003.PS.d7',
 '4.29.2004.PS.100.fptm',
 '5.29.2004.PS.d7',
 '6.3.2004.PQj.d7aconinue',
 '6.16.2004.PS.50a',
 '11.22.2003.PS.100',
 '9.16.2004.PS.100',
 '6.16.2004.PS.50b',
 '7.19.2004.PS.20',
 '7.15.2003.PS.d3',
 '3.3.2004.PQ.d7',
 '5.6.2004.PQ.d7',
 '5.30.2004.PQ.d2complete',
 '5.5.2004.PQ.100',
 '4.29.2004.PS.50',
 '3.9.2004.PQ.d3',
 '8.2.2004.PQ.d7',
 '7.19.2004.PS.100',
 '9.13.2004.PS.d3',
 '6.14.2004.PQ.d3',
 '6.2.2004.PS.2',
 '5.29.2004.PS.5a',
 '8.25.2004.PS.d2',
 '10.8.2003.PQ.d3',
 '3.2.2004.PQ.d2',
 '8.19.2003.PQ.d2',
 '9.14.2003.PS.10complGH',
 '9.28.2004.PS.100',

In [131]:
project_ids = os.listdir(PROJECT_FOLDER)
len(project_ids)

61

In [139]:
# Which ones are in project_ids, but not in ids_in_labelfile? 
missing = [x for x in project_ids if x not in ids_in_labelfile]
missing

# and there was no csv for this file

['6.21.2004.PQ.d7a']

In [140]:
# any weird stuff that shouldnt be there?
weird = [x for x in ids_in_labelfile if x not in project_ids]
weird

[]

In [142]:
#full_labelfile.to_pickle('/Volumes/MaraMeerkat/coati_feeding.pkl')
full_labelfile.to_pickle('/Volumes/EAS_ind/mthomas/analysis/coati_feeding/coati_feeding.pkl')

In [149]:
raw_audios = full_labelfile.raw_audio
callIDs = full_labelfile.callID

missing_callIDs = []

for raw_audio, callID in zip(raw_audios, callIDs):
    if raw_audio.shape[0] == 1:
        missing_callIDs.append(callID)

In [153]:
failed_df = full_labelfile.loc[full_labelfile['callID'].isin(missing_callIDs),:]

In [154]:
# not sure why some of these failed
print(failed_df.shape)
failed_df = failed_df.loc[failed_df.duration_s>0,:]
failed_df.shape

(1042, 11)


(987, 11)

In [ ]:
# some because of zero duration

In [155]:
failed_df

,Name,Start,Duration,Time Format,Type,Description,start_s,duration_s,callID,raw_audio,project_id
1,chirp,5:49.443,0:00.084,decimal,Cue,NaN,349.443,0.084,10.8.2003.PQ.d3_349.443,[0.0],10.8.2003.PQ.d3
2,chirp,5:48.000,0:00.086,decimal,Cue,NaN,348.000,0.086,10.8.2003.PQ.d3_348.0,[0.0],10.8.2003.PQ.d3
3,chirp,5:44.600,0:00.078,decimal,Cue,NaN,344.600,0.078,10.8.2003.PQ.d3_344.6,[0.0],10.8.2003.PQ.d3
4,chirp,5:40.742,0:00.073,decimal,Cue,NaN,340.742,0.073,10.8.2003.PQ.d3_340.742,[0.0],10.8.2003.PQ.d3
5,chirp,5:26.713,0:00.086,decimal,Cue,NaN,326.713,0.086,10.8.2003.PQ.d3_326.713,[0.0],10.8.2003.PQ.d3
...,...,...,...,...,...,...,...,...,...,...,...
77,chirp,0:05.553,0:00.083,decimal,Cue,NaN,5.553,0.083,11.20.2003.PS.10b_5.553,[0.0],11.20.2003.PS.10b
78,chirp,0:05.392,0:00.053,decimal,Cue,NaN,5.392,0.053,11.20.2003.PS.10b_5.392,[0.0],11.20.2003.PS.10b
79,chirp,0:05.278,0:00.062,decimal,Cue,NaN,5.278,0.062,11.20.2003.PS.10b_5.2780000000000005,[0.0],11.20.2003.PS.10b
80,chirp,0:04.000,0:00.045,decimal,Cue,NaN,4.000,0.045,11.20.2003.PS.10b_4.0,[0.0],11.20.2003.PS.10b


In [ ]:
failed_df.reset_index(inplace=True)

In [165]:
failed_df

,level_0,index,Name,Start,Duration,Time Format,Type,Description,start_s,duration_s,callID,raw_audio,project_id
0,0,1,chirp,5:49.443,0:00.084,decimal,Cue,NaN,349.443,0.084,10.8.2003.PQ.d3_349.443,[0.0],10.8.2003.PQ.d3
1,1,2,chirp,5:48.000,0:00.086,decimal,Cue,NaN,348.000,0.086,10.8.2003.PQ.d3_348.0,[0.0],10.8.2003.PQ.d3
2,2,3,chirp,5:44.600,0:00.078,decimal,Cue,NaN,344.600,0.078,10.8.2003.PQ.d3_344.6,[0.0],10.8.2003.PQ.d3
3,3,4,chirp,5:40.742,0:00.073,decimal,Cue,NaN,340.742,0.073,10.8.2003.PQ.d3_340.742,[0.0],10.8.2003.PQ.d3
4,4,5,chirp,5:26.713,0:00.086,decimal,Cue,NaN,326.713,0.086,10.8.2003.PQ.d3_326.713,[0.0],10.8.2003.PQ.d3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,982,77,chirp,0:05.553,0:00.083,decimal,Cue,NaN,5.553,0.083,11.20.2003.PS.10b_5.553,[0.0],11.20.2003.PS.10b
983,983,78,chirp,0:05.392,0:00.053,decimal,Cue,NaN,5.392,0.053,11.20.2003.PS.10b_5.392,[0.0],11.20.2003.PS.10b
984,984,79,chirp,0:05.278,0:00.062,decimal,Cue,NaN,5.278,0.062,11.20.2003.PS.10b_5.2780000000000005,[0.0],11.20.2003.PS.10b
985,985,80,chirp,0:04.000,0:00.045,decimal,Cue,NaN,4.000,0.045,11.20.2003.PS.10b_4.0,[0.0],11.20.2003.PS.10b


In [167]:
missing_audio = {}

# try extracting again

for i in range(failed_df.shape[0]):
    c_id = failed_df.loc[i,'callID']
    p_id =  failed_df.loc[i,'project_id']
    wav_loc = PROJECT_FOLDER+p_id+"/"+p_id+".wav"
    start_s = failed_df.loc[i,'start_s']
    duration_s = failed_df.loc[i,'duration_s']
    try:
        data, rate = librosa.load(wav_loc, offset=start_s, duration=duration_s, sr=None)
        missing_audio[c_id] = data
    except:
        pass

In [168]:
print(len(missing_audio))

987


In [169]:
corrected_audio = [missing_audio[x] if x in missing_audio.keys() else y for x,y in zip(full_labelfile.callID, full_labelfile.raw_audio)]

In [170]:
len(corrected_audio)

7671

In [172]:
full_labelfile['raw_audio'] = corrected_audio

/Users/marathomas/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [173]:
raw_audios = full_labelfile.raw_audio
callIDs = full_labelfile.callID

missing_callIDs = []

for raw_audio, callID in zip(raw_audios, callIDs):
    if raw_audio.shape[0] == 1:
        missing_callIDs.append(callID)

In [176]:
failed_df = full_labelfile.loc[full_labelfile['callID'].isin(missing_callIDs),:]

In [177]:
failed_df

,Name,Start,Duration,Time Format,Type,Description,start_s,duration_s,callID,raw_audio,project_id
0,end,5:49.680,0:00.000,decimal,Cue,NaN,349.680,0.0,10.8.2003.PQ.d3_349.68,[0.0],10.8.2003.PQ.d3
8,sixth coati leaves frame,5:18.254,0:00.000,decimal,Cue,NaN,318.254,0.0,10.8.2003.PQ.d3_318.254,[0.0],10.8.2003.PQ.d3
12,sixth coati leaves grapes,5:10.159,0:00.000,decimal,Cue,NaN,310.159,0.0,10.8.2003.PQ.d3_310.159,[0.0],10.8.2003.PQ.d3
19,second coati runs from frame,4:50.818,0:00.000,decimal,Cue,looks like they are chasing a coati who is out...,290.818,0.0,10.8.2003.PQ.d3_290.818,[0.0],10.8.2003.PQ.d3
28,sixth coati walked to grapes,4:36.575,0:00.000,decimal,Cue,NaN,276.575,0.0,10.8.2003.PQ.d3_276.575,[0.0],10.8.2003.PQ.d3
29,sixth coati enter frame,4:35.274,0:00.000,decimal,Cue,NaN,275.274,0.0,10.8.2003.PQ.d3_275.274,[0.0],10.8.2003.PQ.d3
30,unk third coati left frame,4:35.012,0:00.000,decimal,Cue,NaN,275.012,0.0,10.8.2003.PQ.d3_275.012,[0.0],10.8.2003.PQ.d3
32,unk third coati walks in frame,4:31.820,0:00.000,decimal,Cue,NaN,271.820,0.0,10.8.2003.PQ.d3_271.82,[0.0],10.8.2003.PQ.d3
46,"fith coati enter frame, first coati walks away",4:09.722,0:00.000,decimal,Cue,fith coati runs to grapes,249.722,0.0,10.8.2003.PQ.d3_249.722,[0.0],10.8.2003.PQ.d3
50,first coati turns back to grapes,3:58.621,0:00.000,decimal,Cue,doesnt reach the grapes again,238.621,0.0,10.8.2003.PQ.d3_238.621,[0.0],10.8.2003.PQ.d3


In [ ]:
# these are all not calls!!

In [178]:
full_labelfile.to_pickle('/Volumes/EAS_ind/mthomas/analysis/coati_feeding/coati_feeding.pkl')

In [179]:
full_labelfile.to_pickle('/Volumes/MaraMeerkat/coati_feeding.pkl')

In [180]:
full_labelfile.shape

(7671, 11)

In [ ]:
# this would work if errors are fixed

for project_id, subfolder in zip(project_ids, subfolders):
    labelfile = pd.read_csv(subfolder+project_id+".csv", sep="\t")
    wav_loc = subfolder+project_id+".wav"

    labelfile['start_s'] = labelfile.apply(lambda row: get_s(get_time(row['Start'])), axis=1)
    labelfile['duration_s'] = labelfile.apply(lambda row: get_s(get_time(row['Duration'])), axis=1)
    labelfile['callID'] = [project_id+'_'+str(x) for x in labelfile.start_s]

    raw_audio = labelfile.apply(lambda row: get_audio(wav_loc, # location of long audio file
                                                      row['start_s'], # start of call in that file
                                                      row['duration_s']), # duration of call
                                axis=1)

    labelfile['raw_audio'] = raw_audio

In [183]:
full_labelfile.Name.value_counts()[full_labelfile.Name.value_counts()>3]

chirp                      5380
chitter                    1610
big chitter                 207
squeal chitter              164
squeal                       87
chirpgr                      40
bop                          20
long chirp                   14
chittering not in frame      11
snarl                        10
bark                         10
squawk                        7
vibrate chitter               7
unk bird                      7
Name: Name, dtype: int64